In [1]:
import pandas as pd
import itertools
import numpy as np

### DimPlayerSeasonProfile

In [2]:
values = ['Good', 'Average', 'Bad']

combinations = list(itertools.product(values, repeat=4))


primary_keys = list(range(1, len(combinations) + 1))


df = pd.DataFrame(combinations, columns=['ExperienceLvl', 'OffensiveLvl', 'DefensiveLvl', 'EfficiencyLvl'])
df.insert(0, 'ProfileID', primary_keys)

df.to_csv('GeneratedData/PlayerSeasonProfile.csv', index=False)

### FactPlayerSeason

In [75]:
FactPlayerSeason = pd.read_csv('data/all_seasons.csv').iloc[:, 1:]

In [76]:
FactPlayerSeason = FactPlayerSeason[['player_name', 'team_abbreviation','season', 'age', 'gp', 'pts', 'reb', 'ast', 'net_rating', 'oreb_pct', 'dreb_pct', 'usg_pct', 'ts_pct', 'ast_pct']]

In [77]:
salaries = pd.read_csv('data/salaries_1985to2018.csv').iloc[:, 2:]
players = pd.read_csv('data/players.csv')

In [78]:
salaries = pd.merge(salaries, players, left_on=['player_id'], right_on=['_id'], how='inner')

In [79]:
joindf = salaries[['name','season', 'salary', 'season_end', 'season_start']]

In [80]:
FactPlayerSeason = pd.merge(FactPlayerSeason, joindf, left_on=['player_name', 'season'], right_on=['name','season'], how='inner')
FactPlayerSeason = FactPlayerSeason[['player_name', 'team_abbreviation', 'season_start','season_end','salary', 'age', 'gp', 'pts',
       'reb', 'ast', 'net_rating', 'oreb_pct', 'dreb_pct', 'usg_pct', 'ts_pct']]

In [81]:
FactPlayerSeason = FactPlayerSeason.sort_values(by=['player_name', 'season_start'], ascending=[True, True])

In [82]:
players_names = np.unique(FactPlayerSeason['player_name'])

### DimPlayerInfo

In [83]:
players = pd.read_csv('data/players.csv')

In [84]:
primary_keys = list(range(1, len(players_names) + 1))

DimPlayerInfo = pd.DataFrame(players_names, columns=['PlayerName'])
DimPlayerInfo.insert(0, 'PlayerID', primary_keys)

In [85]:
DimPlayerInfo = pd.merge(DimPlayerInfo, players, left_on=['PlayerName'], right_on=['name'], how='inner')

In [86]:
DimPlayerInfo = DimPlayerInfo[['PlayerID', 'PlayerName', 'birthDate', 'birthPlace', 'draft_pick', 'draft_round', 'draft_year', 'college', 'height', 'weight', 'shoots', 'position']]

In [33]:
DimPlayerInfo['birthPlace'] = DimPlayerInfo['birthPlace'].str.split(',').str[1].str.strip()

In [34]:
stany_usa = [
    'Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware',
    'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky',
    'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri',
    'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York',
    'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island',
    'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
    'West Virginia', 'Wisconsin', 'Wyoming'
]

In [35]:
DimPlayerInfo['birthPlace'] = DimPlayerInfo['birthPlace'].apply(lambda x: 'USA' if x in stany_usa else x)

In [87]:
duplikaty = DimPlayerInfo.groupby('PlayerName').count().reset_index()

In [88]:
duplikaty = duplikaty[duplikaty['PlayerID'] > 1]['PlayerName']

In [89]:
DimPlayerInfo = DimPlayerInfo[~DimPlayerInfo['PlayerName'].isin(duplikaty)]

In [90]:
DimPlayerInfo.loc[DimPlayerInfo['PlayerName'] == 'Ben Bentil', 'birthPlace'] = 'Ghana'
DimPlayerInfo.loc[DimPlayerInfo['PlayerName'] == 'Cheikh Samb', 'birthPlace'] = 'Senegal'
DimPlayerInfo.loc[DimPlayerInfo['PlayerName'] == 'Kornel David', 'birthPlace'] = 'Hungary'
DimPlayerInfo.loc[DimPlayerInfo['PlayerName'] == 'Vladimir Stepania', 'birthPlace'] = 'Georgia'

In [95]:
DimPlayerInfo.groupby(['position']).count()

,PlayerID,PlayerName,birthDate,birthPlace,draft_pick,draft_round,draft_year,college,height,weight,shoots
position,,,,,,,,,,,
Center,266,266,266,266,211,211,211,189,266,266,266
Center and Power Forward,95,95,95,95,85,85,85,85,95,95,95
Center and Power Forward and Small Forward,1,1,1,1,1,1,1,1,1,1,1
Center and Small Forward and Power Forward,3,3,3,3,3,3,3,2,3,3,3
Point Guard,249,249,249,249,169,169,169,230,249,249,249
Point Guard and Power Forward and Small Forward and Shooting Guard,1,1,1,1,1,1,1,0,1,1,1
Point Guard and Shooting Guard,80,80,80,80,64,64,64,76,80,80,80
Point Guard and Shooting Guard and Small Forward,1,1,1,1,1,1,1,1,1,1,1
Point Guard and Small Forward,1,1,1,1,0,0,0,1,1,1,1


In [91]:
DimPlayerInfo.to_csv('GeneratedData/PlayerInfo.csv', index = False)

In [47]:
playerskeys = DimPlayerInfo.iloc[:,0:2]

In [48]:
FactPlayerSeason = pd.merge(FactPlayerSeason, playerskeys, left_on=['player_name'], right_on=['PlayerName'], how='inner')

In [50]:
FactPlayerSeason = FactPlayerSeason[['PlayerID', 'team_abbreviation', 'season_start', 'salary', 'age', 'gp', 'pts', 'reb', 'ast', 'net_rating', 'oreb_pct', 'dreb_pct', 'usg_pct','ts_pct']]

### DimTeam

In [59]:
len(FactPlayerSeason['team_abbreviation'].unique())

30

In [53]:
pd.options.mode.chained_assignment = None  # default='warn'

In [54]:
FactPlayerSeason['team_abbreviation'] = FactPlayerSeason['team_abbreviation'].apply(lambda x: 'BKN' if x in ['NJN'] else x)

In [55]:
FactPlayerSeason['team_abbreviation'] = FactPlayerSeason['team_abbreviation'].apply(lambda x: 'MEM' if x in ['VAN'] else x)

In [56]:
FactPlayerSeason['team_abbreviation'] = FactPlayerSeason['team_abbreviation'].apply(lambda x: 'CHA' if x in ['NOK', 'CHH'] else x)

In [57]:
FactPlayerSeason['team_abbreviation'] = FactPlayerSeason['team_abbreviation'].apply(lambda x: 'OKC' if x in ['SEA'] else x)

In [58]:
FactPlayerSeason['team_abbreviation'] = FactPlayerSeason['team_abbreviation'].apply(lambda x: 'NOP' if x in ['NOH'] else x)

In [60]:
team = pd.read_csv('data/team.csv')
team

,id,full_name,abbreviation,nickname,city,state,year_founded
0,1610612737,Atlanta Hawks,ATL,Hawks,Atlanta,Atlanta,1949.0
1,1610612738,Boston Celtics,BOS,Celtics,Boston,Massachusetts,1946.0
2,1610612739,Cleveland Cavaliers,CLE,Cavaliers,Cleveland,Ohio,1970.0
3,1610612740,New Orleans Pelicans,NOP,Pelicans,New Orleans,Louisiana,2002.0
4,1610612741,Chicago Bulls,CHI,Bulls,Chicago,Illinois,1966.0
5,1610612742,Dallas Mavericks,DAL,Mavericks,Dallas,Texas,1980.0
6,1610612743,Denver Nuggets,DEN,Nuggets,Denver,Colorado,1976.0
7,1610612744,Golden State Warriors,GSW,Warriors,Golden State,California,1946.0
8,1610612745,Houston Rockets,HOU,Rockets,Houston,Texas,1967.0
9,1610612746,Los Angeles Clippers,LAC,Clippers,Los Angeles,California,1970.0


In [61]:
FactPlayerSeason = pd.merge(FactPlayerSeason, team, left_on=['team_abbreviation'], right_on=['abbreviation'], how='inner')

In [64]:
FactPlayerSeason = FactPlayerSeason[['PlayerID', 'id', 'season_start', 'salary', 'age', 'gp',
       'pts', 'reb', 'ast', 'net_rating', 'oreb_pct', 'dreb_pct', 'usg_pct',
       'ts_pct']]

In [65]:
FactPlayerSeason.rename(columns={'id': 'TeamID', 'season_start': 'Season', 'salary': 'Salary'}, inplace=True)

In [66]:
primary_keys = list(range(1, len(FactPlayerSeason) + 1))
FactPlayerSeason.insert(0, 'PlayerSeasonID', primary_keys)

In [67]:
FactPlayerSeason = FactPlayerSeason.sort_values(by=['PlayerID', 'Season'], ascending=[True, True])

In [72]:
FactPlayerSeason[['age']] =FactPlayerSeason[['age']].astype(int)

In [74]:
FactPlayerSeason.to_csv('GeneratedData/PlayerSeason.csv', index = False)